# Preparing a dataset

* How does the dataset handle invalid values?
* What do we want to do with null values?
* Do we want to summarise group or filter the data?

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Diabetes.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [2]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# 데이터프레임에 null 값이 있다면 모두 0으로 채우기
df = df.fillna(0)

In [4]:
# df의 일부 컬럼만을 모아서 df2로 지정
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]

In [5]:
df2.head()

,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


In [6]:
df2.describe()

,Glucose,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000
mean,120.894531,31.992578,33.240885,0.348958
std,31.972618,7.884160,11.760232,0.476951
min,0.000000,0.000000,21.000000,0.000000
25%,99.000000,27.300000,24.000000,0.000000
50%,117.000000,32.000000,29.000000,0.000000
75%,140.250000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


In [11]:
# 0인 값은 사실 null 값으로 제거해야 함
# 컬럼의 값 중 0인 값을 indexing을 통해 찾기 > True인 것은 0이라는 말
df2[df2.columns[:-1]] == 0

,Glucose,BMI,Age
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
763,False,False,False
764,False,False,False
765,False,False,False
766,False,False,False


In [13]:
(df2[df2.columns[:-1]] == 0).any(axis=1)
# any 함수는 주어진 iterable(예: 리스트, 튜플, 세트 등)에 하나 이상의 요소가 `True`로 평가될 경우 `True`를 반환하는 내장 함수입니다.
# 만약 모든 요소가 `False`로 평가되거나 iterable이 비어 있다면 `False`를 반환합니다.

0      False
1      False
2      False
3      False
4      False
       ...  
763    False
764    False
765    False
766    False
767    False
Length: 768, dtype: bool

In [18]:
df3 = df2.loc[~(df2[df2.columns[:-1]] == 0).any(axis=1)] # True 값이 없는 것만 찾도록 loc
df3.describe()

,Glucose,BMI,Age,Outcome
count,752.000000,752.000000,752.000000,752.000000
mean,121.941489,32.454654,33.312500,0.351064
std,30.601198,6.928926,11.709395,0.477621
min,44.000000,18.200000,21.000000,0.000000
25%,99.750000,27.500000,24.000000,0.000000
50%,117.000000,32.300000,29.000000,0.000000
75%,141.000000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


In [19]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 752 entries, 0 to 767
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Glucose  752 non-null    int64  
 1   BMI      752 non-null    float64
 2   Age      752 non-null    int64  
 3   Outcome  752 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 29.4 KB


In [20]:
df3.groupby('Outcome').mean() # 전체를 Outcome으로 구분하여 mean 구하기

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.876434,31.309426
1,142.488636,35.371970,37.015152


In [22]:
df3.groupby('Outcome').agg({"Glucose": "mean"}) # 지정해서 mean

,Glucose
Outcome,
0,110.825820
1,142.488636


In [23]:
df3.groupby('Outcome').agg({"Glucose": "mean", "BMI": "median", "Age": "sum"})


,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.10,15279
1,142.488636,34.25,9772


In [25]:
df3.groupby('Outcome').agg(["mean", "median"])

Glucose               BMI               Age       
               mean median       mean median       mean median
Outcome                                                       
0        110.825820  107.5  30.876434  30.10  31.309426   27.0
1        142.488636  140.5  35.371970  34.25  37.015152   36.0

In [28]:
# Outcome을 두 그룹으로 나누어서 저장한 후, 확인
positive = df3.loc[df3["Outcome"] == 1]
negative = df3.loc[df3["Outcome"] == 0]
print(positive.shape, negative.shape)

(264, 4) (488, 4)


In [29]:
df3.to_csv("clean_diabetes.csv", index=False)